In [1]:
import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os.path

from os import getcwd
import numpy as np
import pandas as pd
import hdf5storage
import h5py
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import savemat, loadmat
import math
from scipy.fftpack import fft, ifft
from scipy.stats import zscore, f_oneway
#import samplerate
import multiprocessing as mp
from scipy.signal import resample
import random
import pyreadr
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import r2_score
import statsmodels.api as sm
import scipy
import matplotlib.patches as mpatches
import statsmodels.stats.multitest as multitest
import mne
from mne.preprocessing import ICA
from mne.time_frequency import tfr_morlet
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [64]:
mer_loc = pd.read_csv('mer_loc.csv', index_col=0).iloc[22:].reset_index(drop=True)
mer_loc.shape
mer_loc.head()

(207, 16)

,SubjectID,session,electrode,unit_id,unit,RecType,grade,Native_X,Native_Y,Native_Z,MNI_X,MNI_Y,MNI_Z,position,Depth,Side
0,DBS3003,1,micro_c,1,1,MU,C,-9.879691,-16.664330,5.719564,-15.143592,-14.083570,-7.384501,c,1.889,left
1,DBS3003,1,micro_p,2,1,MU,C,-9.913828,-18.653425,5.925315,-15.050268,-15.797849,-6.197266,p,1.889,left
2,DBS3003,1,micro_m,3,1,SU,A,-7.986556,-16.630193,6.363670,-13.245183,-13.666481,-6.942032,m,1.889,left
3,DBS3003,1,micro_m,4,2,MU,C,-7.986556,-16.630193,6.363670,-13.245183,-13.666481,-6.942032,m,1.889,left
4,DBS3003,1,micro_m,5,3,SU,A,-7.986556,-16.630193,6.363670,-13.245183,-13.666481,-6.942032,m,1.889,left


In [65]:
cue_unit = pd.read_csv('cue_unit.csv', index_col=0).iloc[22:].reset_index(drop=True)
cue_unit.head()
cue_unit.shape

,SubjectID,Good?,Cue,Cue dir,Cue how,Speech,Speech dir,Speech how,Unnamed: 9,session,electrode,unit_id,unit,RecType,grade
0,DBS3003,NaN,NaN,NaN,NaN,Before,+,All,stim_vol,1,micro_c,1,1,MU,C
1,DBS3003,NaN,NaN,NaN,NaN,Before,+,All,NaN,1,micro_p,2,1,MU,C
2,DBS3003,NaN,After,+,All,Before,+,All,NaN,1,micro_m,3,1,SU,A
3,DBS3003,NaN,NaN,NaN,NaN,Before,+,All,NaN,1,micro_m,4,2,MU,C
4,DBS3003,Good,After,-,All,Before,+,All,NaN,1,micro_m,5,3,SU,A


(207, 15)

In [66]:
save_stn_loc_all = mer_loc.iloc[:, -6:-3]
    
save_stn_loc_all['Subject'] = mer_loc['SubjectID'].factorize()[0]

save_stn_loc_all.shape
save_stn_loc_all.head()

save_stn_loc_all.to_csv('STN_loc/save_stn_loc_all.csv')

(207, 4)

,MNI_X,MNI_Y,MNI_Z,Subject
0,-15.143592,-14.083570,-7.384501,0
1,-15.050268,-15.797849,-6.197266,0
2,-13.245183,-13.666481,-6.942032,0
3,-13.245183,-13.666481,-6.942032,0
4,-13.245183,-13.666481,-6.942032,0


In [67]:
a_0_s_0 = []
a_p_s_0 = []
a_n_s_0 = []

a_0_s_p = []
a_p_s_p = []
a_n_s_p = []

a_0_s_n = []
a_p_s_n = []
a_n_s_n = []

for item in range(cue_unit.shape[0]):
    #if cue_unit.iloc[item, 2] == "After":
        #if cue_unit.iloc[item, 5] == "Before":
        if cue_unit.iloc[item, 3] != "Mixed":
            if cue_unit.iloc[item, 6] == "+":
                if cue_unit.iloc[item, 3] == "+":
                    a_p_s_p.append(item)
                elif cue_unit.iloc[item, 3] == "-":
                    a_n_s_p.append(item)
                else:
                    a_0_s_p.append(item)
    
            elif cue_unit.iloc[item, 6] == "-":
                if cue_unit.iloc[item, 3] == "+":
                    a_p_s_n.append(item)
                elif cue_unit.iloc[item, 3] == "-":
                    a_n_s_n.append(item)
                else:
                    a_0_s_n.append(item)
    
            else:
                if cue_unit.iloc[item, 3] == "+":
                    a_p_s_0.append(item)
                elif cue_unit.iloc[item, 3] == "-":
                    a_n_s_0.append(item)
                else:
                    a_0_s_0.append(item)


a_0_s_0 = mer_loc.iloc[a_0_s_0, -6:-3]
a_p_s_0 = mer_loc.iloc[a_p_s_0, -6:-3]
a_n_s_0 = mer_loc.iloc[a_n_s_0, -6:-3]

a_0_s_p = mer_loc.iloc[a_0_s_p, -6:-3]
a_p_s_p = mer_loc.iloc[a_p_s_p, -6:-3]
a_n_s_p = mer_loc.iloc[a_n_s_p, -6:-3]

a_0_s_n = mer_loc.iloc[a_0_s_n, -6:-3]
a_p_s_n = mer_loc.iloc[a_p_s_n, -6:-3]
a_n_s_n = mer_loc.iloc[a_n_s_n, -6:-3]

a_0_s_0.shape
a_p_s_0.shape
a_n_s_0.shape

a_0_s_p.shape
a_p_s_p.shape
a_n_s_p.shape

a_0_s_n.shape
a_p_s_n.shape
a_n_s_n.shape

(77, 3)

(9, 3)

(13, 3)

(27, 3)

(27, 3)

(14, 3)

(13, 3)

(8, 3)

(15, 3)

In [68]:
a_0_s_0.to_csv('STN_loc/save_stn_loc_a_0_s_0.csv')
a_p_s_0.to_csv('STN_loc/save_stn_loc_a_p_s_0.csv')
a_n_s_0.to_csv('STN_loc/save_stn_loc_a_n_s_0.csv')

a_0_s_p.to_csv('STN_loc/save_stn_loc_a_0_s_p.csv')
a_p_s_p.to_csv('STN_loc/save_stn_loc_a_p_s_p.csv')
a_n_s_p.to_csv('STN_loc/save_stn_loc_a_n_s_p.csv')

a_0_s_n.to_csv('STN_loc/save_stn_loc_a_0_s_n.csv')
a_p_s_n.to_csv('STN_loc/save_stn_loc_a_p_s_n.csv')
a_n_s_n.to_csv('STN_loc/save_stn_loc_a_n_s_n.csv')

In [69]:
a_star_s_0 = pd.concat([a_p_s_0, a_n_s_0], axis=0)
a_0_s_star = pd.concat([a_0_s_p, a_0_s_n], axis=0)
a_star_s_star = pd.concat([a_p_s_p, a_n_s_p, a_p_s_n, a_n_s_n], axis=0)

a_star_s_0.to_csv('STN_loc/save_stn_loc_a_star_s_0.csv')
a_0_s_star.to_csv('STN_loc/save_stn_loc_a_0_s_star.csv')
a_star_s_star.to_csv('STN_loc/save_stn_loc_a_star_s_star.csv')

In [70]:
sig_diff_c = np.array([28, 48, 122, 176, 184, 185, 189, 199])-22

sig_diff_v = np.array([30, 129, 168, 199])-22

cue_unit_c = mer_loc.iloc[sig_diff_c, -6:-3]
cue_unit_v = mer_loc.iloc[sig_diff_v, -6:-3]
cue_unit_cv = pd.concat([cue_unit_c, cue_unit_v], axis=0)

cue_unit_c.shape
cue_unit_v.shape
cue_unit_cv.shape

cue_unit_c.to_csv('STN_loc/save_stn_loc_a_consonant.csv')
cue_unit_v.to_csv('STN_loc/save_stn_loc_a_vowel.csv')
cue_unit_cv.to_csv('STN_loc/save_stn_loc_a_cv.csv')


(8, 3)

(4, 3)

(12, 3)

In [71]:
unique_stn_mni = [[-14.98387934, -12.92955243,  -5.49369781],
       [-14.70173477, -12.72993921,  -6.9148277 ],
       [-14.61559584, -13.08360543,  -6.69066031],
       [-14.30137752, -15.52726452,  -6.05530294],
       [-14.03616376, -13.9636122 ,  -8.70867027],
       [-14.00005879, -14.74497848,  -8.96395878],
       [-13.73481012, -13.47723165,  -5.39605118],
       [-13.56196624, -15.93644309,  -7.66050696],
       [-13.37473971, -17.41903282,  -6.20454728],
       [-13.31060832, -12.20960832,  -5.93658854],
       [-13.29565498, -15.71861443,  -6.72678909],
       [-13.23565603, -15.46435028,  -8.16041817],
       [-13.18059202, -14.17237787,  -8.08707092],
       [-12.17040858, -14.08359043,  -8.36022743],
       [-12.03716398, -14.73622513,  -7.1264492 ],
       [-10.9304167 , -14.53749422,  -5.21299281]]

unique_stn_mni = pd.DataFrame(unique_stn_mni, columns=cue_unit_c.columns)

unique_stn_mni.shape
unique_stn_mni.head()

unique_stn_mni.to_csv('STN_loc/save_stn_loc_stn_stim_to_stg.csv')


(16, 3)

,MNI_X,MNI_Y,MNI_Z
0,-14.983879,-12.929552,-5.493698
1,-14.701735,-12.729939,-6.914828
2,-14.615596,-13.083605,-6.690660
3,-14.301378,-15.527265,-6.055303
4,-14.036164,-13.963612,-8.708670


In [63]:
mer_loc

,SubjectID,session,electrode,unit_id,unit,RecType,grade,Native_X,Native_Y,Native_Z,MNI_X,MNI_Y,MNI_Z,position,Depth,Side
22,DBS3003,1,micro_c,1,1,MU,C,-9.879691,-16.664330,5.719564,-15.143592,-14.083570,-7.384501,c,1.889,left
23,DBS3003,1,micro_p,2,1,MU,C,-9.913828,-18.653425,5.925315,-15.050268,-15.797849,-6.197266,p,1.889,left
24,DBS3003,1,micro_m,3,1,SU,A,-7.986556,-16.630193,6.363670,-13.245183,-13.666481,-6.942032,m,1.889,left
25,DBS3003,1,micro_m,4,2,MU,C,-7.986556,-16.630193,6.363670,-13.245183,-13.666481,-6.942032,m,1.889,left
26,DBS3003,1,micro_m,5,3,SU,A,-7.986556,-16.630193,6.363670,-13.245183,-13.666481,-6.942032,m,1.889,left
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,DBS3032,2,micro_c,4,1,MU,C,-12.321505,-15.500814,3.310374,-12.531485,-13.027212,-7.741866,c,0.910,left
225,DBS3032,2,micro_c,6,3,MU,B,-12.321505,-15.500814,3.310374,-12.531485,-13.027212,-7.741866,c,0.910,left
226,DBS3032,3,micro_c,10,2,MU,B,-12.022031,-16.169508,2.225641,-12.185466,-13.727605,-8.998178,c,-0.399,left
227,DBS3032,3,micro_p,11,1,MU,C,-12.149851,-17.884098,3.247328,-12.229430,-15.471691,-7.700987,p,-0.399,left


In [73]:
sig_beta_supp = np.array([26, 30, 48, 56, 64, 117, 129, 131, 148, 153, 160, 162, 171, 188, 199])-22


sig_beta_supp = mer_loc.iloc[sig_beta_supp, -6:-3]

sig_beta_supp.shape
sig_beta_supp.head()

sig_beta_supp.to_csv('STN_loc/save_stn_loc_beta_supp.csv')


(15, 3)

,MNI_X,MNI_Y,MNI_Z
4,-13.245183,-13.666481,-6.942032
8,-12.730574,-14.414626,-8.574100
26,-14.602097,-15.164714,-5.274902
34,-13.030229,-15.165072,-10.319532
42,-13.540730,-15.273898,-6.055201


In [79]:
cue_speech_encoding_unit = pd.read_csv("cue_speech_encoding_unit.csv", index_col='Unnamed: 0').iloc[22:].reset_index(drop=True)

speech_unit_cv = cue_speech_encoding_unit[cue_speech_encoding_unit['cv_encoding_speech']==1].index

speech_unit_cv = mer_loc.iloc[speech_unit_cv, -6:-3]

speech_unit_cv.shape

speech_unit_cv.to_csv('STN_loc/save_stn_loc_s_cv.csv')


(36, 3)

In [80]:
speech_unit_cv

,MNI_X,MNI_Y,MNI_Z
4,-13.245183,-13.666481,-6.942032
5,-13.245183,-13.666481,-6.942032
6,-14.531058,-16.556697,-7.833296
8,-12.730574,-14.414626,-8.574100
10,-12.730574,-14.414626,-8.574100
14,-11.960274,-15.502574,-10.902627
16,-15.846143,-16.204385,-6.606658
20,-15.723873,-18.262737,-5.897891
26,-14.602097,-15.164714,-5.274902
38,-13.453444,-13.672805,-7.297390
